## Faiss (Facebook AI Similarity Search)

In [2]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter

loader=TextLoader("speech.txt")
documents=loader.load()
text_splitter=CharacterTextSplitter(chunk_size=1000,chunk_overlap=30)
docs=text_splitter.split_documents(documents)

In [3]:
docs

[Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\nâ€¦'),
 Document(metadata={'source': 'speech.txt'}, page_content='â€¦\n\nIt will be all the easier for us to conduct

In [5]:
embeddings=OllamaEmbeddings(model="gemma:2b")
db=FAISS.from_documents(docs,embeddings)
db

In [8]:
## querying
query="How does the speaker describe the desired outcome of the war?"
docs=db.similarity_search(query)
docs[0].page_content

'â€¦\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between usâ€”however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'

In [10]:
retriever=db.as_retriever()
docs=retriever.invoke(query)
docs[0].page_content

'â€¦\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between usâ€”however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'

### Similarity Search with score

In [11]:
docs_and_score=db.similarity_search_with_score(query)
docs_and_score

[(Document(id='752d5ba6-d88c-4bea-b62a-50fbfc5a5fc9', metadata={'source': 'speech.txt'}, page_content='â€¦\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between usâ€”however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'),
  np.float32(2980.2056)),
 (Document(id='c2be61fd-3984-47f1-9198-db8bc7967570', metadata={'source': 'speech.txt'}, page_content='We have borne with their present government through all these bitter months because of 

In [12]:
embedding_vector=embeddings.embed_query(query)
embedding_vector

[0.32234418392181396,
 -2.151080846786499,
 0.2402992695569992,
 0.9927642345428467,
 0.6756510734558105,
 0.5619269013404846,
 -0.8798749446868896,
 0.1038631945848465,
 -0.1124439612030983,
 -0.7927529215812683,
 1.112449288368225,
 -0.02635185420513153,
 -1.0640523433685303,
 1.0723496675491333,
 0.06976828724145889,
 -1.0007189512252808,
 3.045511484146118,
 1.7604950666427612,
 1.20771062374115,
 0.7531377673149109,
 0.3284253478050232,
 -0.28875458240509033,
 0.3017480969429016,
 1.349609375,
 0.23379415273666382,
 -0.4362153112888336,
 -1.3691915273666382,
 -1.2787418365478516,
 -0.5552463531494141,
 -2.0895183086395264,
 -0.2531527876853943,
 -1.440474033355713,
 1.2264454364776611,
 -0.9420737624168396,
 -0.42833665013313293,
 -0.2715860605239868,
 1.8669946193695068,
 0.5418673753738403,
 0.12609539926052094,
 -0.5520623326301575,
 0.3615410029888153,
 0.49248045682907104,
 0.9450090527534485,
 -1.3073257207870483,
 -1.3845535516738892,
 0.6342465281486511,
 0.048859808593988

In [13]:
db.similarity_search_by_vector(embedding_vector)

[Document(id='752d5ba6-d88c-4bea-b62a-50fbfc5a5fc9', metadata={'source': 'speech.txt'}, page_content='â€¦\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between usâ€”however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'),
 Document(id='c2be61fd-3984-47f1-9198-db8bc7967570', metadata={'source': 'speech.txt'}, page_content='We have borne with their present government through all these bitter months because of that friendshipâ€”exercising

In [14]:
## Saving and Loading
db.save_local("faiss_index")

In [16]:
new_db=FAISS.load_local("faiss_index",embeddings,allow_dangerous_deserialization=True)

In [17]:
docs=new_db.similarity_search(query)

In [18]:
docs

[Document(id='752d5ba6-d88c-4bea-b62a-50fbfc5a5fc9', metadata={'source': 'speech.txt'}, page_content='â€¦\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between usâ€”however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'),
 Document(id='c2be61fd-3984-47f1-9198-db8bc7967570', metadata={'source': 'speech.txt'}, page_content='We have borne with their present government through all these bitter months because of that friendshipâ€”exercising